In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:29: SyntaxWarning: invalid escape sequence '\s'
<>:29: SyntaxWarning: invalid escape sequence '\s'
C:\Users\himan\AppData\Local\Temp\ipykernel_18916\972397224.py:29: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [6]:
df2 = normalize_text(df)

In [7]:
df = df2

In [8]:
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [9]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [10]:
df.head()

,sentiment,content
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
6,sadness,sleep im not thinking old friend want he s mar...
8,sadness,charviray charlene love miss
9,sadness,kelcouch i m sorry least friday


In [11]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

C:\Users\himan\AppData\Local\Temp\ipykernel_18916\1089524538.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
C:\Users\himan\AppData\Local\Temp\ipykernel_18916\1089524538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want he s mar...
8,0,charviray charlene love miss
9,0,kelcouch i m sorry least friday


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()


In [19]:
X = cv.fit_transform(df['content'].values).toarray()

In [20]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [21]:
y = df['sentiment']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
import mlflow
mlflow.set_tracking_uri('https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow')



import dagshub
dagshub.init(repo_owner='HimanshuBhardwaj174', repo_name='mlops-project', mlflow=True)

mlflow.set_experiment('LogisticRegressionBaselineModel')

Accessing as HimanshuBhardwaj174

Initialized MLflow to track repo "HimanshuBhardwaj174/mlops-project"

Repository HimanshuBhardwaj174/mlops-project initialized!

2024/07/29 21:24:31 INFO mlflow.tracking.fluent: Experiment with name 'LogisticRegressionBaselineModel' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/73b194eaadd743ddbb2d396b5404957b', creation_time=1722268470431, experiment_id='0', last_update_time=1722268470431, lifecycle_stage='active', name='LogisticRegressionBaselineModel', tags={}>

In [26]:
with mlflow.start_run():
    
    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)
    
    # Model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log model
    mlflow.sklearn.log_model(model, "model")

    # Save and log the notebook
    notebook_path = "exp_1.ipynb"
    
    mlflow.log_artifact(notebook_path)
    
    # Print the results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

2024/07/29 21:29:50 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Accuracy: 0.7946987951807228
Precision: 0.7862001943634597
Recall: 0.7970443349753694
F1 Score: 0.7915851272015656


2024/07/29 21:29:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run valuable-pug-519 at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/0/runs/64c0a6c1ae3948338b937762f65e7f26.
2024/07/29 21:29:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/0.


## Experiment -2


In [32]:
import mlflow
mlflow.set_tracking_uri('https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow')



import dagshub
dagshub.init(repo_owner='HimanshuBhardwaj174', repo_name='mlops-project', mlflow=True)

mlflow.set_experiment('bow+allML')

Initialized MLflow to track repo "HimanshuBhardwaj174/mlops-project"

Repository HimanshuBhardwaj174/mlops-project initialized!

<Experiment: artifact_location='mlflow-artifacts:/edececb2470243618e8c62c31abe20f4', creation_time=1722269962031, experiment_id='1', last_update_time=1722269962031, lifecycle_stage='active', name='bow+allML', tags={}>

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

# Start the parent run
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                # X = vectorizer.fit_transform(df['content'])
                # y = df['sentiment']
                # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                

                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                
                # Model training
                model = algorithm
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                # Log model
                mlflow.sklearn.log_model(model, "model")
                
                # Save and log the notebook
                
                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

2024/07/29 22:16:54 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.7946987951807228
Precision: 0.7862001943634597
Recall: 0.7970443349753694
F1 Score: 0.7915851272015656


2024/07/29 22:16:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression with BoW at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1/runs/c141b5d87da147ee9c8330f0c5570495.
2024/07/29 22:16:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1.
2024/07/29 22:17:16 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Algorithm: LogisticRegression, Feature Engineering: TF-IDF
Accuracy: 0.7946987951807228
Precision: 0.7862001943634597
Recall: 0.7970443349753694
F1 Score: 0.7915851272015656


2024/07/29 22:17:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1/runs/7430d86a55184c4b9e02d349443b486c.
2024/07/29 22:17:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1.
2024/07/29 22:17:31 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


...

Algorithm: XGBoost, Feature Engineering: BoW
Accuracy: 0.7590361445783133
Precision: 0.7180355630821338
Recall: 0.8354679802955665
F1 Score: 0.7723132969034608


2024/07/29 22:18:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBoost with BoW at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1/runs/98156836f2c846928c6dd8244a6b1cd8.
2024/07/29 22:18:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1.
2024/07/29 22:18:45 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Algorithm: XGBoost, Feature Engineering: TF-IDF
Accuracy: 0.7590361445783133
Precision: 0.7180355630821338
Recall: 0.8354679802955665
F1 Score: 0.7723132969034608


2024/07/29 22:18:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBoost with TF-IDF at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1/runs/319dd00574344b57881a73bebb1e87da.
2024/07/29 22:18:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1.
2024/07/29 22:21:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest with BoW at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1/runs/9016eba08ff24262893757453ed9b44d.
2024/07/29 22:21:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1.
2024/07/29 22:21:28 INFO mlflow.tracking._tracking_service.client: 🏃 View run All Experiments at: https://dagshub.com/HimanshuBhardwaj174/mlops-project.mlflow/#/experiments/1/runs/b11b53dd7fb441c0b974fc319b3940ae.
2024/07/29 22:

KeyboardInterrupt: 